# <font color='lightblue'>
Information Retrieval Project - Jigsaw Rate Severity of Toxic Comments

MohammadReza 
</font>

In [1]:
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re

In [151]:
comments_to_score = pd.read_csv('./datasets/comments_to_score.csv')
leaderboard = pd.read_csv('./datasets/leaderboard.csv')
sample_submission = pd.read_csv('./datasets/sample_submission.csv')
validation_data = pd.read_csv('./datasets/validation_data.csv')

In [143]:
# t1 = validation_data['less_toxic'].iloc[9]
# t2 = validation_data['more_toxic'].iloc[9]

# determine_severity_toxicity(22, t1)
# determine_severity_toxicity(22, t2)

# print(model.predict(t1))
# print(model.predict(t2))

# print('\n\n',t1)
# print('\n----\n', t2)


### <font color='lightgreen'> Detoxify library</font>

In [9]:
import detoxify

# Load the pre-trained model
model = detoxify.Detoxify('original')

h:\Anaconda - File\installed\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mohammadreza\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## <font color='yellow'>1. Detect_toxicity </font>

In [27]:
def detect_toxicity(text):
    possibility_labels = model.predict(text)
    return sum(possibility_labels.values())

## <font color='yellow'>2. Analyze_intent </font>

In [171]:
# Exapmle:
from nltk.sentiment import SentimentIntensityAnalyzer
# Create a SentimentIntensityAnalyzer object
sid = SentimentIntensityAnalyzer()

sentiment_scores = sid.polarity_scores('you are idiot black asshole')
# sentiment_scores = sid.polarity_scores('I love  you')
print(sentiment_scores)  # 'compound': -0.5106 _ it's negative, therefore we use asb()

{'neg': 0.452, 'neu': 0.548, 'pos': 0.0, 'compound': -0.5106}


In [106]:
def analyze_intent(text):
    # Get the sentiment scores
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['neg']

## <font color='yellow'>3. Count_toxic_words </font>

In [54]:
with open('toxic_words.txt', 'r') as f:
    toxic_words = f.read().split()

In [116]:
def count_toxic_words(text):
    count = 0
    seen_words = []
    for word in text.split():
        if word in toxic_words and word not in seen_words:
            count += 0.01
            seen_words.append(word)
        elif word in toxic_words and word in seen_words:
            count += 0.0
    return count


## <font color='yellow'> Preprocess </font>

In [186]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]',' ', text)
################################################################################################
def lower_case(text):
    return text.lower()

################################################################################################
def remove_numbers(text):
    res = ''
    for char in text:
     if not char.isdigit():
          res += char
          
    return res

################################################################################################
with open('stopwords.txt', 'r') as stopwords_file:
    stopwords = stopwords_file.read().split()

def remove_stopwords(text):
    # Tokenize the input text
    word_tokens = nltk.word_tokenize(text)

    # Remove the stop words
    filtered_text = [word for word in word_tokens if not word in stopwords]

    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_text)
    
    return filtered_text

################################################################################################
def tokenize(text):
    return nltk.word_tokenize(text)
################################################################################################

lemmatizer = WordNetLemmatizer()

In [185]:
comments_to_score_temp = comments_to_score.copy()

In [187]:
comments_to_score_temp['text'] = comments_to_score_temp['text'].apply(remove_punctuation)
comments_to_score_temp['text'] = comments_to_score_temp['text'].apply(lower_case)
comments_to_score_temp['text'] = comments_to_score_temp['text'].apply(remove_numbers)
comments_to_score_temp['text'] = comments_to_score_temp['text'].apply(remove_stopwords)
comments_to_score_temp['text'] = comments_to_score_temp['text'].apply(tokenize)
comments_to_score_temp['text'] = comments_to_score_temp['text'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])


In [65]:
lemma = lambda tokens: [lemmatizer.lemmatize(token) for token in tokens]

In [68]:
def preprocess_text(text):
    text = remove_punctuation(text)
    text = lower_case(text)
    text = remove_numbers(text)
    text = remove_stopwords(text)
    text = tokenize(text)
    text = lemma(text)
    # Join the filtered words back into a string
    text = ' '.join(text)
    return text
    

In [180]:
def toxicity(token):
    return model.predict(token)['toxicity']

In [181]:
def find_toxic_words():
    tokens = []
    for i, row in comments_to_score_temp.iterrows():
        tokens.extend(row['text'])
        
    tokens = list(set(tokens))
    
    toxic_words = []
    for token in tokens:
        if toxicity(token) >= 0.20:
            toxic_words.append(token)
    
    return toxic_words

In [188]:
toxic_words = find_toxic_words()


In [189]:
print(len(toxic_words))

1818


In [190]:
# write toxic_words list to file
with open('toxic_words.txt', 'w') as fp:
    for tw in toxic_words:
        fp.write(tw + '\n')

In [191]:
with open('toxic_words.txt', 'r') as f:
    toxic_words = f.read().split()

# <font color='yellow'> determine_severity_toxicity </font>

In [160]:
comment_score = []
def determine_severity_toxicity(c_id, text):
    # global comment_id, comment_score
    global comment_score
    
    # Criteria 1: Language and Tone
    toxicity_score = detect_toxicity(text)

    # Criteria 2: Intent
    intent_score = analyze_intent(text)

    # Criteria 3: Frequency and Consistency
    toxic_words_count = count_toxic_words(preprocess_text(text))

    # Calculate an overall toxicity score for each text based on the criteria
    overall_score = (toxicity_score + intent_score + toxic_words_count) / 3

   
    comment_score.append((c_id, overall_score))
    # print(c_id, '  score: ', overall_score)


## <font color='yellow'>Calling determine_severity_toxicity: </font>

In [192]:
for i, row in comments_to_score.iterrows():
    determine_severity_toxicity(row['comment_id'], row['text'])


In [193]:
comment_score_final = pd.DataFrame(comment_score, columns=['comment_id', 'score'])
comment_score_final

,comment_id,score
0,114890,0.023445
1,732895,0.059164
2,1139051,0.019165
3,1434512,0.034741
4,2084821,0.185361
...,...,...
15069,504235362,0.314145
15070,504235566,0.024135
15071,504308177,0.074069
15072,504570375,0.765043


In [194]:
comment_score_final = comment_score_final.sort_values(by='score', ascending=False)
comment_score_final

,comment_id,score
14625,459378692,2.188433
7088,459378692,2.188433
334,28808212,2.008952
7871,28808212,2.008952
333,28807993,1.974345
...,...,...
14769,473244297,0.000440
12956,316890379,0.000440
5419,316890379,0.000440
13418,349203625,0.000438


In [195]:
comment_score_final.to_csv('comment_score_final.csv', index=False)

In [196]:
df = pd.read_csv('comment_score_final.csv')
df

,comment_id,score
0,459378692,2.188433
1,459378692,2.188433
2,28808212,2.008952
3,28808212,2.008952
4,28807993,1.974345
...,...,...
15069,473244297,0.000440
15070,316890379,0.000440
15071,316890379,0.000440
15072,349203625,0.000438


### <font color='lightblue'>Finish</font>